In [1]:
from distribution import TestDistribution

from RTER import RegressionTree

import numpy as np

from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

from time import time
import os




In [2]:


#distribution_index_vec=[1,2,3,4]





np.random.seed(1)



sample_generator=TestDistribution(7).returnDistribution()
n_test, n_train = 1000,1000
X_train, Y_train = sample_generator.generate(n_train)
X_test, Y_test = sample_generator.generate(n_test)



# RTER with cv
time_start=time()
parameters={"truncate_ratio_low":[0], "truncate_ratio_up":[1 ],
           "min_samples_split":[10,30], "max_depth":[1,2,3,4],
           "order":[1,2],"splitter":["maxedge"],
           "r_range_low":[0,0.1],"r_range_up":[0.5,0.6,0.7]}
cv_model_RTER=GridSearchCV(estimator=RegressionTree(),param_grid=parameters, cv=7, n_jobs=-1)
cv_model_RTER.fit(X_train, Y_train)

RTER_model = cv_model_RTER.best_estimator_
mse_score= - RTER_model.score(X_test, Y_test)
y_hat_RTER = RTER_model.predict(X_test)

time_end=time()

print("RTER: mse {} time {}".format(mse_score,time_end-time_start))


# GBRT
time_start=time()

parameters= {"n_estimators":[500,1000,2000], "learning_rate":[0.01,0.05]}
cv_model_GBRT=GridSearchCV(estimator=GradientBoostingRegressor(),param_grid=parameters, cv=7, n_jobs=-1)
cv_model_GBRT.fit(X_train, Y_train)
model_GBRT = cv_model_GBRT.best_estimator_
model_GBRT.fit(X_train, Y_train.ravel())

y_hat=model_GBRT.predict(X_test)
mse_score = MSE(y_hat, Y_test)

time_end=time()

print("GBRT: mse {} time {}".format(mse_score,time_end-time_start))


# RF
time_start=time()

model_RFR = RandomForestRegressor(n_estimators = 10)
model_RFR.fit(X_train, Y_train.ravel())

y_hat=model_RFR.predict(X_test)
mse_score = MSE(y_hat, Y_test)

time_end=time()

print("RF: mse {} time {}".format(mse_score,time_end-time_start))

RTER: mse 0.1292148941424587 time 46.72032642364502
GBRT: mse 0.12078640612922806 time 5.724526882171631
RF: mse 0.13387030141898065 time 0.04139971733093262


In [3]:
cv_model_RTER.best_params_

{'max_depth': 2,
 'min_samples_split': 10,
 'order': 1,
 'r_range_low': 0,
 'r_range_up': 0.6,
 'splitter': 'maxedge',
 'truncate_ratio_low': 0,
 'truncate_ratio_up': 1}